## Investigation of Windows Login Attempts

## Import

In [9]:
import pandas as pd

print(pd.__version__)

2.3.3


## Load File

In [10]:
def load_logs(path: str) -> pd.DataFrame:
    """Loads and returns the JSONL log dataframe.
        
        Args:
            path (str): Path to the log file.

        Returns:
            pd.DataFrame: Loaded data.
    """
    print(f'Reading file: {path}')
    df = pd.read_json(path, lines=True)
    print(f'Total number of lines: {len(df)}')
    return df

df = load_logs('../data_raw/WinEvents.log')
df.head()

Reading file: ../data_raw/WinEvents.log
Total number of lines: 338


,EventTime,Hostname,Keywords,EventType,SeverityValue,Severity,EventID,SourceName,ProviderGuid,Version,...,StateTransition,PreviousState,PreviousStateName,NewStateName,Event,EventName,AppPoolID,Minutes,Title,FileList
0,2021-10-02T18:58:47.573161+05:45,DC01.corp.local,9.232379e+18,AUDIT_SUCCESS,2,INFO,5140,Microsoft-Windows-Security-Auditing,{54849625-5478-4994-A5BA-3E3B0328C30D},1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-10-02T18:57:15.277442+05:45,DC01.corp.local,9.232379e+18,AUDIT_SUCCESS,2,INFO,5145,Microsoft-Windows-Security-Auditing,{54849625-5478-4994-A5BA-3E3B0328C30D},0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-10-02T18:59:05.463675+05:45,DC01.corp.local,9.232379e+18,AUDIT_SUCCESS,2,INFO,4672,Microsoft-Windows-Security-Auditing,{54849625-5478-4994-A5BA-3E3B0328C30D},0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-10-02T18:59:05.463675+05:45,DC01.corp.local,9.232379e+18,AUDIT_SUCCESS,2,INFO,4624,Microsoft-Windows-Security-Auditing,{54849625-5478-4994-A5BA-3E3B0328C30D},1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-10-02T18:58:55.314065+05:45,IT02.corp.local,9.232379e+18,AUDIT_SUCCESS,2,INFO,4670,Microsoft-Windows-Security-Auditing,{54849625-5478-4994-A5BA-3E3B0328C30D},0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Filtering Logon Events (4624 and 4625)

In [16]:
def filter_logon_events(df: pd.DataFrame) -> pd.DataFrame:
    """Filter out logon-related events (4624 and 4625).

    Args:
       df (pd.DataFrame): Original Dataframe.

    Returns:
        pd.DataFrame: Filtered dataframe.
    """

    df_logon = df[df['EventID'].isin([4624, 4625])].copy()

    print(f'Login events found: {len(df_logon)}')

    return df_logon

df_logon = filter_logon_events(df)
df_logon.head()

Login events found: 2


,EventTime,Hostname,Keywords,EventType,SeverityValue,Severity,EventID,SourceName,ProviderGuid,Version,...,StateTransition,PreviousState,PreviousStateName,NewStateName,Event,EventName,AppPoolID,Minutes,Title,FileList
3,2021-10-02T18:59:05.463675+05:45,DC01.corp.local,9.232379e+18,AUDIT_SUCCESS,2,INFO,4624,Microsoft-Windows-Security-Auditing,{54849625-5478-4994-A5BA-3E3B0328C30D},1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
232,2021-09-21T13:28:43.986770+05:45,ACC01.prod.corp.local,9.227876e+18,AUDIT_FAILURE,4,ERROR,4625,Microsoft-Windows-Security-Auditing,{54849625-5478-4994-A5BA-3E3B0328C30D},0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Event Statistics

In [12]:
print('Count by EventID:')
print(df_logon['EventID'].value_counts())
print()

if 'TargetUserName' in df_logon.columns:
    print(df_logon['TargetUserName'].value_counts().head(10))

Count by EventID:
EventID
4624    1
4625    1
Name: count, dtype: int64

TargetUserName
DC01$            1
administrator    1
Name: count, dtype: int64


## Separation between Successes and Failures

In [13]:
success_df = df_logon[df_logon['EventID'] == 4624]
fail_df = df_logon[df_logon['EventID'] == 4625]

print(f'Sucess: {len(success_df)} | Fail: {len(fail_df)}')

Sucess: 1 | Fail: 1


## Analysis of a Failure Event

In [14]:
if not fail_df.empty:
    print('\nExample of a failure:')
    columns_of_interest = ['TargetUserName', 'SubjectUserName', 'UserID', 'User', 'IpAddress', 'IpPort', 'LogonType']

    for col in columns_of_interest:
        if col in fail_df.columns:
            print(f'{col}: {fail_df[col].iloc[0]}')
            
else:
    print('No failure events found.')


Example of a failure:
TargetUserName: administrator
SubjectUserName: -
UserID: nan
User: nan
IpAddress: 172.16.20.11
IpPort: 0.0
LogonType: 3.0
